In [293]:
import random
import pandas as pd
import numpy as np
from dataclasses import dataclass, field
from itertools import product

In [294]:
RNG = np.random.default_rng()

DIM = 3
LAYER = ["L0", "L1", "L2"]
SCALE = [1,1,150,150,150,150,15,15,15,15]

def gen_part_name():
    _prefix = "TM"
    _suffix = f'{random.randint(0,99):02d}'
    return _prefix + chr(random.randint(65,90)) + chr(random.randint(65,90)) + _suffix

def gen_ret_name(part_name, layer_name, num):
    return [f'{part_name}{layer_name}-{i}' for i in range(1, num+1)]

def gen_bias(dim, scale, base=None):
    _scale = np.array(scale[:DIM]).reshape([DIM, 1])
    if base is None:
        bias = 10 * (1/_scale) * RNG.random(size=[dim, 1])
    else:
        bias = base + 10 * (1/_scale) * RNG.random(size=[dim, 1]) / 10        
    return bias

In [295]:
@dataclass
class Part:
    id: int
    n_ret: list

    name: str=field(init=False)
    ret_name: dict=field(init=False)
    bias: list=field(init=False)
    ret_bias: dict=field(init=False)
    chuck_type: list=field(init=False)

    def __post_init__(self):
        self.name = gen_part_name()
        self.bias = gen_bias(DIM, SCALE)
        self.ret_name = dict()
        self.ret_bias = dict()

        for layer, num  in zip(LAYER, self.n_ret):
            ret_names = gen_ret_name(self.name, layer, num)
            self.ret_name[layer] = ret_names

            for ret_name in ret_names:
                self.ret_bias[ret_name] = gen_bias(DIM, SCALE, base=self.bias)

        self.chuck_type = random.choice([["BASE"], ["S1","S2"]])


@dataclass
class Tool:
    id: int
    
    name: str=field(init=False)
    bias0: list=field(init=False)
    bias1: list=field(init=False)
    bias2: list=field(init=False)
    
    chuck0: list=field(init=False)
    chuck1: list=field(init=False)
    chuck2: list=field(init=False)
    
    def __post_init__(self):
        self.name = f'Tool-{self.id}'
        self.bias0 = gen_bias(DIM, SCALE)
        self.bias1 = gen_bias(DIM, SCALE)
        self.bias2 = gen_bias(DIM, SCALE)
        
        self.chuck0 = gen_bias(DIM, SCALE)
        self.chuck1 = gen_bias(DIM, SCALE)
        self.chuck2 = gen_bias(DIM, SCALE)

In [ ]:
pp = Part(1, [2,1,2])
pp

In [ ]:
tt = Tool(1)
tt

In [298]:
def gen_data(n_part, n_ret, n_tool):
    _part_map = [Part(i, n_ret) for i in range(n_part)]
    _tool_map = [Tool(i) for i in range(n_tool)]
    
    part_map = {part.name: part for part in _part_map}
    tool_map = {tool.name: tool for tool in _tool_map}
    return part_map, tool_map

n_part = 5
n_ret = [2,1,1]
n_tool = 10

part_map, tool_map = gen_data(n_part, n_ret, n_tool)

In [ ]:
LST_FEAT = ["Part", "Ret0", "Ret1", "Tool0", "Tool1", "Tool2", "Chuck"]

lst_comb = []
for part_name, part in part_map.items():
    lst_tool = list(tool_map.keys())
    lst_feats = [[part_name], part.ret_name["L0"], part.ret_name["L1"], lst_tool[:5], lst_tool[5:], lst_tool[5:], part.chuck_type]
    lst_comb.extend(list(product(*lst_feats)))

df_data = pd.DataFrame(lst_comb, columns=LST_FEAT)
df_data


In [ ]:
def cal_rn(row):
    _Part = row["Part"]
    _Ret0 = row["Ret0"]
    _Ret1 = row["Ret1"]
    _Tool0 = row["Tool0"]
    _Tool1 = row["Tool1"]
    _Tool2 = row["Tool2"]
    _Chuck = row["Chuck"]
    
    rn0 = part_map[_Part].ret_bias[_Ret0][0] + tool_map[_Tool0].bias0[0]
    rn1 = part_map[_Part].ret_bias[_Ret1][0] + tool_map[_Tool1].bias1[0]
    rn2 = tool_map[_Tool2].bias2[0] + 0
    
    if _Chuck != "BASE":
        rn0 += tool_map[_Tool0].chuck0[0]
        rn1 += tool_map[_Tool1].chuck1[0]
        rn2 += tool_map[_Tool2].chuck2[0]
        
    return float(rn0+rn1+rn2)
    
    
df_data["Tx"] = df_data.apply(cal_rn, axis=1)
df_data

In [ ]:
tool_map['Tool-4']